In [1]:
# ============================================================
# COCO(val2017) × YOLOv5s 평가 파이프라인
# - mAP@0.5, mAP@0.5:0.95
# - 간단 Precision/Recall (IoU=0.5, greedy 1:1 매칭)
# - Latency/FPS (샘플 64장)
# - 모델 크기(바이트) / 파라미터 수
# - 학습시간(기본 0, 옵션: 짧게 학습하고 측정 가능)
# ============================================================

import os, time, json, math, random, warnings, glob
import numpy as np
import pandas as pd
import cv2
from pathlib import Path

warnings.filterwarnings("ignore", category=FutureWarning,
                        message=r".*torch\.cuda\.amp\.autocast.*")

# -----------------------------
# 0) 설정
# -----------------------------
ROOT = Path("/content")
COCO_DIR = ROOT/"coco"
ANN_JSON = COCO_DIR/"annotations"/"instances_val2017.json"
VAL_DIR  = COCO_DIR/"val2017"          # val 이미지 폴더
OUTPUT   = ROOT/"yolov5s_eval_out"     # 로그 저장 경로
OUTPUT.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "yolov5s"
IMG_SIZE = 640
CONF_THR = 0.25
IOU_THR  = 0.50
LAT_SAMPLES = 64
DO_TRAIN = False  # True로 놓으면 아주 짧게 학습을 돌리고 학습시간 측정(권장X: 시간이 걸립니다)

MAP_CSV    = OUTPUT/"map.csv"
PR_CSV     = OUTPUT/"pr.csv"
LAT_TXT    = OUTPUT/"latency.json"
SIZE_TXT   = OUTPUT/"size.json"
TRAIN_TIME = OUTPUT/"train_time.txt"

# -----------------------------
# 1) 필수 설치 & 데이터 준비
# -----------------------------
!pip -q install --upgrade pip
!pip -q install pycocotools opencv-python torch torchvision --index-url https://download.pytorch.org/whl/cu121
!git -C /content clone -q https://github.com/ultralytics/yolov5.git || true
!pip -q install -r /content/yolov5/requirements.txt

# COCO val2017 다운로드(이미 받았다면 건너뜁니다)
if not ANN_JSON.exists():
    COCO_DIR.mkdir(parents=True, exist_ok=True)
    # 어노테이션
    !mkdir -p /content/coco/annotations
    !wget -qO /content/coco/annotations_trainval2017.zip http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip -q /content/coco/annotations_trainval2017.zip -d /content/coco
    # val 이미지
    !wget -qO /content/coco/val2017.zip http://images.cocodataset.org/zips/val2017.zip
    !unzip -q /content/coco/val2017.zip -d /content/coco

print("COCO paths ready:", ANN_JSON.exists(), VAL_DIR.exists())

# -----------------------------
# 2) YOLOv5s 로드
# -----------------------------
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
yolo = torch.hub.load('ultralytics/yolov5', MODEL_NAME, pretrained=True, source='github', force_reload=True).to(DEVICE)
yolo.conf = CONF_THR
yolo.iou  = IOU_THR
yolo.max_det = 300
NAMES = yolo.names  # {0:'person', 1:'bicycle', ...}

# -----------------------------
# 3) COCO API & 카테고리 매핑
# -----------------------------
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
cocoGt = COCO(str(ANN_JSON))

# COCO: name -> category_id (COCO는 1,2,3... 비연속 id)
cats = cocoGt.loadCats(cocoGt.getCatIds())
NAME2CATID = {c['name']: c['id'] for c in cats}
IDX2CATID  = {i: NAME2CATID.get(name, None) for i, name in NAMES.items()}

# -----------------------------
# 4) (옵션) 아주 짧게 학습 돌려서 학습시간 측정
# -----------------------------
train_time_sec = 0.0
if DO_TRAIN:
    import subprocess, shlex
    # ultralytics/yolov5의 train.py를 이용해 COCO의 80클래스로 아주 짧게 학습 (데모용)
    # ※ 실제 COCO 학습은 시간이 오래 걸립니다. 여기서는 epochs=1~3 정도로 데모만 권장
    t0 = time.time()
    cmd = f"python /content/yolov5/train.py --img {IMG_SIZE} --batch 16 --epochs 1 --data coco.yaml --weights {MODEL_NAME}.pt --device 0 --project {OUTPUT} --name train_demo"
    print("Running:", cmd)
    p = subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(p.stdout)
    train_time_sec = time.time() - t0

# -----------------------------
# 5) 예측 수집 → COCO mAP 평가
# -----------------------------
img_ids = cocoGt.getImgIds()
imgs = cocoGt.loadImgs(img_ids)

def run_detector(image_bgr):
    img_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    with torch.no_grad():
        res = yolo(img_rgb, size=IMG_SIZE)
    dets=[]
    if hasattr(res, "xyxy") and len(res.xyxy):
        arr = res.xyxy[0].detach().cpu().numpy()  # x1,y1,x2,y2,conf,cls
        for x1, y1, x2, y2, score, cls in arr:
            cls = int(cls)
            cat_id = IDX2CATID.get(cls)  # COCO category_id로 변환
            if cat_id is None:  # 혹시 이름 매칭 실패 시 스킵
                continue
            dets.append({
                "bbox": [int(x1), int(y1), int(x2), int(y2)],
                "score": float(score),
                "category_id": int(cat_id)
            })
    return dets

preds = []
val_image_paths = []
t_eval0 = time.time()
for im in imgs:
    file_name = im["file_name"]  # e.g., 000000397133.jpg
    img_path = str(VAL_DIR / file_name)
    val_image_paths.append(img_path)
    img = cv2.imread(img_path)
    if img is None:
        continue
    dets = run_detector(img)
    for d in dets:
        x1,y1,x2,y2 = d["bbox"]
        w = max(0, x2 - x1); h = max(0, y2 - y1)
        preds.append({
            "image_id": int(im["id"]),
            "category_id": int(d["category_id"]),
            "bbox": [float(x1), float(y1), float(w), float(h)],  # COCO format [x,y,w,h]
            "score": float(d["score"])
        })
eval_time_sec = time.time() - t_eval0

if len(preds)==0:
    map50 = 0.0; map5095 = 0.0
    print("No predictions collected.")
else:
    cocoDt = cocoGt.loadRes(preds)
    cocoEval = COCOeval(cocoGt, cocoDt, iouType='bbox')
    cocoEval.evaluate(); cocoEval.accumulate(); cocoEval.summarize()
    map5095 = float(cocoEval.stats[0])  # mAP@0.5:0.95
    map50   = float(cocoEval.stats[1])  # mAP@0.5

# -----------------------------
# 6) 간단 Precision / Recall (IoU=0.5)
# -----------------------------
def iou_xyxy(a, b):
    ax1,ay1,ax2,ay2 = a; bx1,by1,bx2,by2 = b
    ix1,iy1 = max(ax1,bx1), max(ay1,by1)
    ix2,iy2 = min(ax2,bx2), min(ay2,by2)
    iw,ih = max(0, ix2-ix1), max(0, iy2-iy1)
    inter = iw*ih
    ua = max(0, ax2-ax1)*max(0, ay2-ay1)
    ub = max(0, bx2-bx1)*max(0, by2-by1)
    union = ua + ub - inter + 1e-9
    return inter/union

# GT 인덱스: image_id -> [boxes]
gt_by_img = {}
for ann in cocoGt.dataset["annotations"]:
    x,y,w,h = ann["bbox"]
    gt_by_img.setdefault(ann["image_id"], []).append([x,y,x+w,y+h])

# Pred 인덱스: image_id -> [(box, score)]
pred_by_img={}
for pr in preds:
    x,y,w,h = pr["bbox"]
    pred_by_img.setdefault(pr["image_id"], []).append(([x,y,x+w,y+h], pr["score"]))

TP=FP=FN=0
for img_id, gts in gt_by_img.items():
    prs = pred_by_img.get(img_id, [])
    matched=set()
    prs = sorted(prs, key=lambda x: -x[1])
    for p_box, _ in prs:
        ok=False
        for j, g_box in enumerate(gts):
            if j in matched:
                continue
            if iou_xyxy(p_box, g_box) >= 0.5:
                TP+=1; matched.add(j); ok=True; break
        if not ok:
            FP+=1
    FN += (len(gts) - len(matched))

precision = TP / (TP+FP+1e-9)
recall    = TP / (TP+FN+1e-9)

# -----------------------------
# 7) Latency/FPS 측정 (샘플 64장)
# -----------------------------
def measure_latency(paths, max_samples=64):
    paths = paths[:max_samples]
    if not paths: return math.nan, math.nan
    # warm-up
    dummy = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
    for _ in range(5): _ = run_detector(dummy)
    if DEVICE == 'cuda': torch.cuda.synchronize()

    times=[]
    for p in paths:
        img = cv2.imread(p)
        if img is None: continue
        t0=time.time()
        _ = run_detector(img)
        if DEVICE == 'cuda': torch.cuda.synchronize()
        times.append((time.time()-t0)*1000.0)
    if not times: return math.nan, math.nan
    avg_ms = float(np.mean(times))
    fps = 1000.0/avg_ms if avg_ms>0 else math.nan
    return avg_ms, fps

random.shuffle(val_image_paths)
avg_ms, fps = measure_latency(val_image_paths, LAT_SAMPLES)

# -----------------------------
# 8) 모델 크기 / 파라미터 수
# -----------------------------
TMP_W = OUTPUT/"yolov5s_tmp_weights.pt"
file_bytes = -1
try:
    torch.save(yolo.model.state_dict(), TMP_W)
    file_bytes = int(TMP_W.stat().st_size)
except Exception:
    pass
num_params = int(sum(p.numel() for p in yolo.model.parameters()))

# -----------------------------
# 9) 로그 저장
# -----------------------------
pd.DataFrame([{"split":"val2017","map50":map50,"map50_95":map5095}]).to_csv(MAP_CSV, index=False)
pd.DataFrame([{"split":"val2017","precision":precision,"recall":recall}]).to_csv(PR_CSV, index=False)
with open(LAT_TXT, "w") as f:
    json.dump({"avg_ms_per_image":avg_ms, "fps":fps, "eval_time_sec":eval_time_sec}, f)
with open(SIZE_TXT, "w") as f:
    json.dump({"file_bytes":file_bytes, "num_params":num_params}, f)
with open(TRAIN_TIME, "w") as f:
    f.write(f"{train_time_sec:.6f}")

print("\n=== SUMMARY ===")
print("mAP@0.5:0.95:", map5095)
print("mAP@0.5    :", map50)
print("Precision  :", precision)
print("Recall     :", recall)
print("Latency(ms):", avg_ms, "| FPS:", fps)
print("Params     :", num_params, "| Weights file size(bytes):", file_bytes)
print("Train time :", train_time_sec, "sec")
print("Logs saved :", str(OUTPUT))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.3 MB/s eta 0:00:00
COCO paths ready: True True


/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-9-2 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 156MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


loading annotations into memory...
Done (t=0.52s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=11.34s).
Accumulating evaluation results...
DONE (t=1.79s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.321
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.481
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.350
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.158
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.372
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.262
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.374
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDet